<a href="https://colab.research.google.com/github/grendelaglaeca/XGBoost-brain-regions/blob/main/xgboost_tutorial_Chris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preliminaries**

Let’s first load the required packages. For any packages that you haven’t yet installed on your system, you can install them one at a time using the function install.packages('packagename').

In [2]:
import timeit

import numpy as np
import pandas as pd
import xgboost as xgb
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Load the data from a CSV file


In [9]:
url = "https://github.com/grendelaglaeca/XGBoost-brain-regions/raw/main/tutorial_all_data_speech.xlsx"
data = pd.read_excel(url)
print(data.shape)
data.head(20)

(2452, 96)


,Filename,Vocality,Emotion,Intensity,Statement,Repetition,Speaker,Sex,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0.2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,loudness_sma3_amean,loudness_sma3_stddevNorm,loudness_sma3_percentile20.0,loudness_sma3_percentile50.0,loudness_sma3_percentile80.0,loudness_sma3_pctlrange0.2,loudness_sma3_meanRisingSlope,loudness_sma3_stddevRisingSlope,loudness_sma3_meanFallingSlope,loudness_sma3_stddevFallingSlope,spectralFlux_sma3_amean,spectralFlux_sma3_stddevNorm,mfcc1_sma3_amean,mfcc1_sma3_stddevNorm,mfcc2_sma3_amean,mfcc2_sma3_stddevNorm,mfcc3_sma3_amean,mfcc3_sma3_stddevNorm,mfcc4_sma3_amean,mfcc4_sma3_stddevNorm,jitterLocal_sma3nz_amean,jitterLocal_sma3nz_stddevNorm,...,F2bandwidth_sma3nz_amean,F2bandwidth_sma3nz_stddevNorm,F2amplitudeLogRelF0_sma3nz_amean,F2amplitudeLogRelF0_sma3nz_stddevNorm,F3frequency_sma3nz_amean,F3frequency_sma3nz_stddevNorm,F3bandwidth_sma3nz_amean,F3bandwidth_sma3nz_stddevNorm,F3amplitudeLogRelF0_sma3nz_amean,F3amplitudeLogRelF0_sma3nz_stddevNorm,alphaRatioV_sma3nz_amean,alphaRatioV_sma3nz_stddevNorm,hammarbergIndexV_sma3nz_amean,hammarbergIndexV_sma3nz_stddevNorm,slopeV0.500_sma3nz_amean,slopeV0.500_sma3nz_stddevNorm,slopeV500.1500_sma3nz_amean,slopeV500.1500_sma3nz_stddevNorm,spectralFluxV_sma3nz_amean,spectralFluxV_sma3nz_stddevNorm,mfcc1V_sma3nz_amean,mfcc1V_sma3nz_stddevNorm,mfcc2V_sma3nz_amean,mfcc2V_sma3nz_stddevNorm,mfcc3V_sma3nz_amean,mfcc3V_sma3nz_stddevNorm,mfcc4V_sma3nz_amean,mfcc4V_sma3nz_stddevNorm,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0.500_sma3nz_amean,slopeUV500.1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
2432,03-02-04-02-01-01-24,2,4,2,1,1,24,f,44.803860,0.036632,43.755340,44.315250,46.523070,2.767731,19.188198,3.504519,7.420797,6.073151,0.201372,1.129384,0.008069,0.143822,0.325152,0.317083,2.406952,1.614991,2.392360,1.476358,0.054167,1.246049,9.482435,1.559625,-2.470419,-4.891333,-11.000483,-1.405482,-26.527199,-0.793516,0.012882,1.229587,...,878.86710,0.457126,-113.318886,-0.768496,2930.2075,0.118660,634.15290,0.664541,-117.511580,-0.700825,-15.177430,-0.707701,27.826490,0.343869,0.176497,0.168987,-0.010388,-1.373839,0.101077,0.670110,20.547482,0.496671,-3.755027,-3.919695,-17.818705,-1.005616,-42.835407,-0.364797,0.508526,6.998731,0.124252,0.006430,0.009222,1.829268,1.437372,0.334286,0.253932,0.338571,0.450315,-41.181730
2433,03-02-04-02-01-02-24,2,4,2,1,2,24,f,44.539604,0.034487,43.607452,43.899204,46.396767,2.789314,11.287774,7.211615,2.726269,8.595187,0.282515,1.049692,0.008910,0.188746,0.507094,0.498184,2.919323,1.426895,3.633452,2.335462,0.068932,1.190931,9.828450,1.294498,-6.025487,-2.117146,-10.241098,-1.658653,-25.822702,-0.866852,0.009009,1.379311,...,831.67584,0.449425,-106.638140,-0.843268,2855.5427,0.128008,692.79980,0.631896,-111.864250,-0.754976,-12.777843,-0.645552,25.147148,0.269606,0.172768,0.145187,-0.008519,-1.064256,0.127925,0.597955,19.652145,0.451922,-8.916692,-1.536514,-19.407460,-0.906394,-43.438050,-0.366489,0.769587,6.230379,0.108768,0.004941,0.007172,1.829268,0.821355,0.625000,0.621953,0.456000,0.496169,-38.377598
2434,03-02-04-02-02-01-24,2,4,2,2,1,24,f,44.716297,0.032614,43.713010,44.169650,46.240437,2.527428,68.816170,114.178160,9.621263,5.793366,0.265044,0.933162,0.007674,0.246703,0.496427,0.488753,1.559141,0.799277,2.855995,1.777271,0.054723,1.058320,10.752875,1.455033,-6.081100,-1.912517,-13.043976,-1.024276,-27.323603,-0.850638,0.008728,1.470348,...,872.81690,0.315231,-98.542920,-0.8975

**Features**

It will be helpful for us to store the names of the acoustic features so that we can access them more easily later on. We know from the summary above that the acoustic features begin at the ninth column in the data set and continue until the very last column, so we can simply extract those column names and verify that we’ve got the right ones:


In [24]:
col_names = list(data)  #gets a list of column headers
features = col_names[8:len(col_names)]  #python starts the count from zero
features

['F0semitoneFrom27.5Hz_sma3nz_amean',
 'F0semitoneFrom27.5Hz_sma3nz_stddevNorm',
 'F0semitoneFrom27.5Hz_sma3nz_percentile20.0',
 'F0semitoneFrom27.5Hz_sma3nz_percentile50.0',
 'F0semitoneFrom27.5Hz_sma3nz_percentile80.0',
 'F0semitoneFrom27.5Hz_sma3nz_pctlrange0.2',
 'F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope',
 'loudness_sma3_amean',
 'loudness_sma3_stddevNorm',
 'loudness_sma3_percentile20.0',
 'loudness_sma3_percentile50.0',
 'loudness_sma3_percentile80.0',
 'loudness_sma3_pctlrange0.2',
 'loudness_sma3_meanRisingSlope',
 'loudness_sma3_stddevRisingSlope',
 'loudness_sma3_meanFallingSlope',
 'loudness_sma3_stddevFallingSlope',
 'spectralFlux_sma3_amean',
 'spectralFlux_sma3_stddevNorm',
 'mfcc1_sma3_amean',
 'mfcc1_sma3_stddevNorm',
 'mfcc2_sma3_amean',
 'mfcc2_sma3_stddevNorm',
 'mfcc3_sma3_amean',
 'mfcc3_sma3_stddevNorm',
 'mfcc4_

These acoustic features were generated with openSMILE, using the GeMAPS feature set (v02). Specifically, these are the high-level, derived parameters (“functionals”) which make up the extended Geneva Minimalistic Acoustic Parameter Set (eGeMAPS). There are 88 parameters per audio file in the data base, which is exactly how many feature names are in the extracted *features* object:

# **XGBoost**
**preparation**

We’ll start by making a copy of the data set (so that anything we add/subtract/change doesn’t mess up the original data):

In [26]:
xgbdata = data

We’re going to set up an XGBoost model as a classification problem (i.e. is it sung or spoken?). For classification problems, XGBoost wants to have ordinal classes starting with 0. Let’s remind ourselves what our current class labels are:

In [27]:
xgbdata.Vocality.value_counts()   #equivalent of R *table* function

1    1440
2    1012
Name: Vocality, dtype: int64

We have an ordinal series (1: spoken, 2: sung) but it starts at 1, so we simply need to subtract 1 from the class series. We’ll put that into a separate column called *levels*:

In [28]:
xgbdata.Vocality

0       1
1       1
2       1
3       1
4       1
       ..
2447    2
2448    2
2449    2
2450    2
2451    2
Name: Vocality, Length: 2452, dtype: int64

In [29]:
xgbdata["levels"] = xgbdata.Vocality-1

In [31]:
xgbdata.levels.value_counts()

0    1440
1    1012
Name: levels, dtype: int64

Nice! Our classes are now ready for XGBoost classification.

One last thing before we start. Many types of calculations and algorithms will contain a random component of some sort. It’s always good practice to ensure that the random components can be replicated (either by yourself or by other researchers), so that your results will always come out the same way in the end. To do this in R, we choose some number (literally, any number!) and then set that as the random “seed” for the session. We’ll use the number 123: